In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
# Generate mock A/B test data
np.random.seed(42)
num_users_per_group = 5000
start_date = datetime(2023, 11, 1)
test_duration_days = 14 # 2 week test

In [ ]:
# Control Group (A)
control_user_ids = [f'UserA_{10000+i}' for i in range(num_users_per_group)]
control_group_assignment = ['Control'] * num_users_per_group
# Assume a baseline conversion rate of 10% for control
control_conversions = np.random.binomial(1, 0.10, num_users_per_group)
control_clicks = np.random.randint(0, 15, num_users_per_group) # Clicks per user
control_page_views = control_clicks + np.random.randint(1, 5, num_users_per_group) # Page views > clicks
control_page_views = np.maximum(1, control_page_views) # At least 1 page view

In [ ]:
# Treatment Group (B) - let's assume a slight lift
treatment_user_ids = [f'UserB_{20000+i}' for i in range(num_users_per_group)]
treatment_group_assignment = ['Treatment'] * num_users_per_group
# Assume a target conversion rate of 11.5% for treatment (1.5% lift)
treatment_conversions = np.random.binomial(1, 0.115, num_users_per_group)
treatment_clicks = np.random.randint(0, 17, num_users_per_group) # Slightly more clicks
treatment_page_views = treatment_clicks + np.random.randint(1, 5, num_users_per_group)
treatment_page_views = np.maximum(1, treatment_page_views)

In [ ]:
# Combine data
all_user_ids = control_user_ids + treatment_user_ids
all_groups = control_group_assignment + treatment_group_assignment
all_conversions = np.concatenate([control_conversions, treatment_conversions])
all_clicks = np.concatenate([control_clicks, treatment_clicks])
all_page_views = np.concatenate([control_page_views, treatment_page_views])

In [ ]:
# Assign random dates within the test period
all_dates = [ (start_date + timedelta(days=np.random.randint(0, test_duration_days))).strftime('%Y-%m-%d')for _ in range(num_users_per_group * 2) ]

In [ ]:
df_ab_test = pd.DataFrame({
    'UserID': all_user_ids,
    'Group': all_groups,
    'Date': all_dates,
    'PageViews': all_page_views,
    'Clicks': all_clicks,
    'Converted': all_conversions
})

In [ ]:
# Ensure clicks are not more than page views (though unlikely with current generation)
df_ab_test['Clicks'] = df_ab_test.apply(lambda row: min(row['Clicks'], row['PageViews']), axis=1)

In [ ]:
#Save to CSV
df_ab_test.to_csv('ab_test_results_mock_data.csv', index=False)

In [ ]:
df=pd.read_csv('ab_test_results_mock_data.csv')

In [ ]:
print(df.sample(10, random_state=42))

In [ ]:
# Duplicate user across group 
duplicates=df.groupby('UserID')['Group'].nunique()
users_in_both_groups=duplicates[duplicates>1]
print(users_in_both_groups)

In [ ]:
# check missing value 
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.boxplot(df['PageViews'])
plt.title('Boxplot of PageViews')
plt.show()

plt.figure(figsize=(6,4))
plt.boxplot(df['Clicks'])
plt.title('Boxplot of Clicks')
plt.show()


In [ ]:
# Ensure correct assignment of users to groups
print(df.groupby('Group')['UserID'].nunique())

In [ ]:
df.head()

In [ ]:
# Control Group Conversion Rate 
control_converted=df[df['Group']== 'Control']['Converted'].sum()
control_total=len(df[df['Group']=='Control'])
control_rate=(control_converted/control_total)*100

In [ ]:
# Treatment Group Conversion Rate 
Treatment_converted=df[df['Group']=='Treatment']['Converted'].sum()
Treatment_total=len(df[df['Group']=='Treatment'])
Treatment_rate=(Treatment_converted/Treatment_total)*100

In [ ]:
print(f'Control Group conversion rate: {control_rate }%')
print(f'Treatment Group conversion rate: {Treatment_rate }%')

In [ ]:
# Difference between two groups 
difference = Treatment_rate-control_rate
print("Difference between control and Treatment Group",difference)

In [ ]:
print(df.head())
print(df.tail())

In [ ]:
from statsmodels.stats.proportion import proportion_confint,proportions_ztest
control_converted,control_total=200,5000
treatment_converted, treatment_total=250,5000

# Z-test 
count=np.array([control_converted,treatment_converted])
nobs=np.array([control_total,treatment_total])
z_stats,p_value=proportions_ztest(count,nobs,alternative='two-sided')

print("P-value",p_value)
if p_value < 0.05:
    print("Result: The difference is significant (treatment is better)!")
else:
    print("Conclusion: The difference is not significant.")


ci_control = proportion_confint(control_converted, control_total, alpha=0.05)
ci_treatment = proportion_confint(treatment_converted, treatment_total, alpha=0.05)
print(f"\n Control CI: {ci_control[0]:.3%} to {ci_control[1]:.3%}")
print(f"Treatment CI: {ci_treatment[0]:.3%} to {ci_treatment[1]:.3%}")

In [ ]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# Parameters
control_rate = 9.58
treatment_rate = 11.79
alpha = 0.05
power = 0.8

# Effect size
effect_size = proportion_effectsize(control_rate, treatment_rate)

# Calculate required sample size per group
analysis = NormalIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1, alternative='two-sided')

print(f"Required sample size per group to detect 1.5% uplift with 80% power: {(sample_size)}")


In [ ]:
print("\n--- Interpretation & Recommendation ---")
if p_value < 0.05:
    print(" Conversion uplift is statistically significant.")
    if difference >= 1.0:
        print(f" Uplift of {difference:.2f}% seems practically meaningful. Consider rolling out the feature.")
    else:
        print(f" Although statistically significant, uplift of {difference:.2f}% may not be practically meaningful. Consider testing further.")
else:
    print(" No statistically significant difference found.")
    print("Suggestion: Either increase test duration/sample size or iterate on the feature.")
